In [ ]:
!pip install --upgrade pinecone
!pip install --upgrade openai

from pinecone import Pinecone, ServerlessSpec
import time
import pandas as pd
import csv
from google.colab import userdata
from openai import OpenAI

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

index_name = "events-index2"
model = "text-embedding-3-small"

In [ ]:
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ),
        deletion_protection="disabled",
        tags={
            "environment": "development"
        }
    )

index = pc.Index(index_name)

# Function to generate vector embeddings
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

# Read CSV and add records to Pinecone
def csv_to_pinecone(csv_file):
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            event_id = hashlib.md5(row['title'].encode()).hexdigest()
            event_text = f"{row['title']} {row['description']} {row['location']} {row['organizer']}"
            embedding = get_embedding(event_text)  # Generate vector

import hashlib
import csv

# Assuming `get_embedding` and other necessary functions are already defined

with open('events.csv', 'r') as file:
    reader = csv.DictReader(file)
    for i, row in enumerate(reader):
        event_id = hashlib.md5(row['title'].encode()).hexdigest()

        # Use the updated field names and combine relevant fields for event text
        event_text = f"{row['title']} {row['details']} {row['location']} {row['hosts']}"

        # Generate embedding for the event
        embedding = get_embedding(event_text)  # Generate vector

        # Map the row to the event data with updated column names
        event_data = {
            "title": row["title"],
            "location": row["location"],
            "image": row["image"],
            "hosts": row["hosts"],
            "main_host": row["main_host"],
            "link": row["link"],
            "description": row["details"],
            "start": int(row["start"]),
            "end": int(row["end"]),
            "perks": row["Perks"],
            "categories": row["categories"],
            "start_day_time": int(row["start_day_time"]),
            "end_day_time": int(row["end_day_time"])
        }

        # You can now process `event_data` or store it as needed
        index.upsert([(event_id, embedding, event_data)])  # Store vector and metadata
        if i % 10 == 0:
            print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650


In [ ]:
# !pip install --upgrade pinecone
# !pip install --upgrade openai

# from pinecone import Pinecone, ServerlessSpec
# import time
# import pandas as pd
# from google.colab import userdata
# from openai import OpenAI

# client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
# pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

# index_name = "events-index"
# index = pc.Index(index_name)
# model = "text-embedding-3-small"

# Function to generate vector embeddings
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

def search(text, top_k=10):
    vector = get_embedding(text)
    return index.query(top_k=top_k, vector=vector, include_metadata=True)

search("game tournament")

{'matches': [{'id': 'b229a1658ceaa68f5646859742e687bb',
              'metadata': {'categories': '',
                           'description': 'Come compete for a prize in a Mario '
                                          'Kart Tournament! Signups in Wagner '
                                          'Group me, walk-ins welcome',
                           'end': 1742436000.0,
                           'hosts': 'Roosevelt Community',
                           'image': 'https://se-images.campuslabs.com/clink/images/54be12c3-97b9-4095-9436-0a36e6862851083acbab-b642-4723-8284-6ef0028e6f9b.png?preset=med-w',
                           'link': 'https://stonybrook.campuslabs.com/engage/event/11062387',
                           'location': 'Wagner Lounge',
                           'main_host': 'Roosevelt Community',
                           'perks': 'Credit',
                           'start': 1742432400.0,
                           'title': 'Mario Kart Tournament with Wagner'},
 

In [ ]:
question = input("What are you looking for?")
context = search(question, top_k=5)
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "developer", "content": "User will give you a question and a RAG system will give you relevant events to that question. Tell them what events are relevant, and let them know if nothing was found."},
    {"role": "user", "content": f"context: {context}\n\nNow, answer this question: {question}"}
  ]
)

print(completion.choices[0].message.content)

What are you looking for?Give me some AI related events to attend in the next month
Here are some upcoming AI-related events you can attend in the next month:

1. **Explore AI Pop Up Series: Prompt Engineering**
   - **Date:** Starts on a Monday (specific date not provided) at 12:30 PM
   - **Location:** Galleria
   - **Description:** Learn about generative AI and prompt engineering with demonstrations from librarians and DoIT staff. They will host Explore AI demos from Monday to Wednesday of this week.
   - **Link:** [Explore AI Pop Up Series: Prompt Engineering](https://stonybrook.campuslabs.com/engage/event/11068488)

2. **Explore AI Pop Up Series: Image Generators**
   - **Date:** March 3 - March 5 (specific time not provided)
   - **Location:** Galleria
   - **Description:** Explore different AI image generation tools and see demonstrations to learn more.
   - **Link:** [Explore AI Pop Up Series: Image Generators](https://stonybrook.campuslabs.com/engage/event/11069313)

3. **GBM 

In [ ]:
pd.read_csv("events.csv")["Perks"].value_counts()

,count
Perks,
Credit,1429
"Free Food, Credit",88
"Free Stuff, Credit",79
"Free Food, Free Stuff, Credit",56
